In [25]:
#May 28 - for ease for getting data out
import numpy as np
import matplotlib.pyplot as pl
import os

In [42]:
def getData(text):
    #get center z
    data = np.loadtxt(text, skiprows=0, max_rows=2, dtype=('<U16', '<U16'), comments="@")
    data = np.transpose(data)
    z = (float(data[1][1]) - float(data[1][0]))/2 + float(data[1][0])
    zstr = str(round(z, 5))

    #check if errors are linear
    data = np.loadtxt(text, skiprows=2, max_rows=1, dtype=('<U16', '<U16'), comments="@")
    data = np.transpose(data)

    linear = False
    if data[1] == 'linear':
        linear = True
        
    #get data
    data = np.loadtxt(text, skiprows=0)
    data = np.transpose(data)

    masses = data[0]
    phi = data[1]
    err_up = data[2]
    err_down = data[3]
    
    if type(err_up) == np.float64:
        masses = np.array([masses])
        phi = np.array([phi])
        err_up = np.array([err_up])
        err_down = np.array([err_down])

    #putting the errors together in pairs
    errs = []

    if linear:

        for i in range(len(err_up)):
            errs.append((np.log10(err_up[i]), np.log10(err_down[i])))
            phi[i] = np.log10(data[1][i]) #need to convert phi values too

    else:    
        for i in range(len(err_up)):
            errs.append((err_up[i], err_down[i]))

    #check if the masses are evenly spaced
    even = True
    if len(masses) > 2:
        i = 1
        massDif = (masses[1] - masses[0])
        while i < (len(masses)-1) and even:
            if massDif != round((masses[i+1] - masses[i]), 14):
                even = False
            i += 1 
    else:
        even = False
    
    #get the masses in the correct format
    if not even: #if they are not even, explicitly in a list in 10** form
        masses = np.array([10**m for m in masses])
        massesStr = "["
        for m in masses:
            massesStr = massesStr + "{:.7E}, ".format(m)
        
        massesStr = massesStr.rsplit(',', 1)[0] + "]"
        
        formatThese = [massesStr, str(phi.tolist()), str(errs)]
    else:
        formatThese = [str(masses.tolist()), str(phi.tolist()), str(errs)]
            
    #space each data type nicely
    Strings = ["", "", ""]
    for i, value in enumerate(formatThese):
        
#         print(value)
#         print(len(value))

        #loop through the line while we cut it as long as it's long enough
        while len(value) > 121 : 
            split = 120
            ending = False
            
            if value[split] == ',':
                noComma = False
            else:
                noComma = True
                
            #increase split point until a comma is found
            while noComma & (not ending):
                split +=1
#                 print(split, value[split])
                if value[split] == ',': #found a comma, proceed to splitting
                    noComma = False
                if len(value[split:].split(',')) == 1 or len(value) < split + 1:
                    ending = True

            if not ending:
                split +=1 #go to the next space
                #make a new line
                Strings[i] = Strings[i] + value[:split] + "\n          "
                value = value[split:]
#                 print(len(value))
            else: #got to end of line
                Strings[i] = Strings[i] + value
                value = "" #to end the while loop as value will have lenght 0
        else: #line is too short to split
            Strings[i] = Strings[i] + value
    
    #writting the data to a string
    if even:
        incr = masses[1] - masses[0]
        minM = masses[0]
        maxM = masses[-1] + incr
        
        pyFile = " {}: {{'M': list(10**np.arange({}, {:.3f}, {:.3f})),\n    'phi': {},\n    'err': {}\n   }},".format(zstr, \
                                                    minM, maxM-incr, incr, Strings[1], Strings[2])
#         print(len(np.arange(minM, maxM-incr, incr)))

    else:
        pyFile = " {}: {{'M': {},\n    'phi': {},\n    'err': {}\n   }},".format(zstr,\
                                                    Strings[0], Strings[1], Strings[2])
#         print(len(masses))

    
    return pyFile, z

In [ ]:
directory = "../behroozi-2013-data-compilation/smf_ms"
names = ["a"]
zArr = []
for filename in os.listdir(directory):
    if filename.endswith(".smf"):
        
        name = os.path.join(directory, filename)
        author = filename.split('_')[0]
        if author != names[-1]:
            print(zArr)
            print(f"\n{author}\n")
            zArr = []
            
        names.append(author)
        data, z = getData(name)
        print(data)   
        zArr.append(round(z, 5))
        
    else:
        continue

In [59]:
file = "../behroozi-2013-data-compilation/smf_ms/marchesini_z1.7.smf"

data, z = getData(file)
# print(data)

In [43]:
#ssfr

directory = "../behroozi-2013-data-compilation/ssfr"
names = ["a"]
zArr = []
for filename in os.listdir(directory):
#     print(filename)
    if filename.endswith(".ssfr"):
        
        name = os.path.join(directory, filename)
        author = filename.split('_')[0]
        if author != names[-1]:
            print(zArr)
            print(f"\n{author}\n")
            zArr = []
            
        names.append(author)
        data, z = getData(name)
        print(data)   
        zArr.append(round(z, 5))
        
    else:
        continue

[]

bouwens

 7.0: {'M': [1.1481536E+09],
    'phi': [-8.1],
    'err': [(0.3, 0.3)]
   },
 8.0: {'M': [1.4454398E+09],
    'phi': [-8.2],
    'err': [(0.3, 0.3)]
   },
[7.0, 8.0]

daddi

 2.0: {'M': [5.8823529E+09, 1.7647059E+10, 5.8823529E+10],
    'phi': [-8.60205999132796, -8.65, -8.69897000433602],
    'err': [(0.3, 0.3), (0.3, 0.3), (0.3, 0.3)]
   },
[2.0]

dunne

 0.5: {'M': [9.3229011E+08, 2.3418069E+09, 5.8823529E+09, 1.4775803E+10, 2.9481602E+10, 5.8823529E+10],
    'phi': [-9.52287874528034, -9.52287874528034, -9.60205999132796, -9.69897000433602, -9.76955107862172, -9.82390874094432],
    'err': [(0.3, 0.3), (0.3, 0.3), (0.3, 0.3), (0.3, 0.3), (0.3, 0.3), (0.3, 0.3)]
   },
 0.95: {'M': [3.7115138E+09, 5.8823529E+09, 1.4775803E+10, 2.9481602E+10, 7.4054436E+10],
    'phi': [-9.0, -9.09691001300805, -9.15490195998574, -9.22184874961636, -9.15490195998574],
    'err': [(0.3, 0.3), (0.3, 0.3), (0.3, 0.3), (0.3, 0.3), (0.3, 0.3)]
   },
 1.4: {'M': [4.6725190E+09, 9.3229011E+09, 

ValueError: Wrong number of columns at line 2

In [16]:
x = np.array(1.0)

print(x)

1.0
